#Step 2: DATA UNDERSTANDING

In [ ]:
import import_ipynb
import pandas as pd

In [ ]:
# create statistics function
def continous_feature_describe(lst_cols, df):
    col_ = []
    count_ = []
    min_ = []
    max_ = []
    mean_ = []
    std_ = []
    var_ = []
    skewness_ = []
    kurtosis_ = []
    do_lech = []
    do_nhon = []

    for i in lst_cols:
        col_.append(i)
        count_.append(df[i].count())
        min_.append(df[i].min())
        max_.append(df[i].max())
        mean_.append(df[i].mean())
        std_.append(df[i].std())
        var_.append(df[i].var())
        skewness_.append(df[i].skew())
        kurtosis_.append(df[i].kurtosis())
        
        if df[i].skew() > 0: do_lech.append('lech phai')
        elif df[i].skew() < 0: do_lech.append('lech trai')
        else: do_lech.append('doi xung')
            
        if df[i].kurtosis() > 0: do_nhon.append('nhon hon PP chuan')
        elif df[i].kurtosis() < 0: do_nhon.append('bet hon PP chuan')
        else: do_nhon.append('tuong duong PP chuan')

    df_describe = pd.DataFrame({'feature_name': col_,
                               'count': count_,
                               'min': min_,
                               'max': max_,
                               'mean': mean_,
                               'std': std_,
                               'var': var_,
                                'skewness': skewness_,
                                'kurtosis': kurtosis_,
                                'do_lech': do_lech,
                                'do_nhon': do_nhon
                               })
    return df_describe

#Step 4&5: MODELING & EVALUATION/ANALYZE

In [ ]:
# create Cross validation:
def Average_RMSE_Model(model, X, y, size=0.3, cv=10):
    import time
    from sklearn.model_selection import train_test_split    
    train_rmse=[]
    train_score=[]
    test_rmse=[]
    test_score=[]
    
    duration=[]
    for n in range(1, cv+1):                
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size)                                             
        
        start=time.time()        
        model.fit(X_train, y_train)
        end=time.time()
        
        train_rmse.append(mean_squared_error(y_true=y_train, y_pred=model.predict(X_train), squared=False))
        test_rmse.append(mean_squared_error(y_true=y_test, y_pred=model.predict(X_test), squared=False))
        train_score.append(model.score(X_train, y_train))
        test_score.append(model.score(X_test, y_test))

        duration.append((end-start)*1000)
        
    return np.mean(train_rmse), np.mean(train_score), np.mean(test_rmse), np.mean(test_score), np.mean(duration) 

In [ ]:
def Save_Object(obj, filename):
    with open(filename, 'wb') as file:
        pickle.dump(obj, file)
    return
def Load_Object(filename):
    with open(filename, 'rb') as file:
        obj = pickle.load(file)
    return obj

## Problem 1

### Create def

In [ ]:
def data_cleaning(data):
    df = data.copy()
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    # drop duplicates
    df = df.drop_duplicates() 
    # change datatype
    df['Date'] = pd.to_datetime(df['Date']) 
    # change column names
    df.columns = df.columns.str.replace(" ","_")
    df.columns = ['Date', 'AveragePrice', 'Total_Volume', 'PLU_4046','PLU_4225','PLU_4770',
          'Total_Bags', 'Small_Bags', 'Large_Bags', 'XLarge_Bags', 'type', 'year',
          'region']
    # remove some region: 
    df = df.loc[df['region'] != "TotalUS",:]
    return df

### Create fit_transform function